# Day 14: Extended Polymerization

[https://adventofcode.com/2021/day/14](https://adventofcode.com/2021/day/14)

## Description

### Part One

The incredible pressures at this depth are starting to put a strain on your submarine. The submarine has [polymerization](https://en.wikipedia.org/wiki/Polymerization) equipment that would produce suitable materials to reinforce the submarine, and the nearby volcanically-active caves should even have the necessary input elements in sufficient quantities.

The submarine manual contains <span title="HO

HO -> OH">instructions</span> for finding the optimal polymer formula; specifically, it offers a _polymer template_ and a list of _pair insertion_ rules (your puzzle input). You just need to work out what polymer would result after repeating the pair insertion process a few times.

For example:

    NNCB

    CH -> B
    HH -> N
    CB -> H
    NH -> C
    HB -> C
    HC -> B
    HN -> C
    NN -> C
    BH -> H
    NC -> B
    NB -> B
    BN -> B
    BB -> N
    BC -> B
    CC -> N
    CN -> C


The first line is the _polymer template_ - this is the starting point of the process.

The following section defines the _pair insertion_ rules. A rule like `AB -> C` means that when elements `A` and `B` are immediately adjacent, element `C` should be inserted between them. These insertions all happen simultaneously.

So, starting with the polymer template `NNCB`, the first step simultaneously considers all three pairs:

*   The first pair (`NN`) matches the rule `NN -> C`, so element _`C`_ is inserted between the first `N` and the second `N`.
*   The second pair (`NC`) matches the rule `NC -> B`, so element _`B`_ is inserted between the `N` and the `C`.
*   The third pair (`CB`) matches the rule `CB -> H`, so element _`H`_ is inserted between the `C` and the `B`.

Note that these pairs overlap: the second element of one pair is the first element of the next pair. Also, because all pairs are considered simultaneously, inserted elements are not considered to be part of a pair until the next step.

After the first step of this process, the polymer becomes `NCNBCHB`.

Here are the results of a few steps using the above rules:

    Template:     NNCB
    After step 1: NCNBCHB
    After step 2: NBCCNBBBCBHCB
    After step 3: NBBBCNCCNBBNBNBBCHBHHBCHB
    After step 4: NBBNBNBBCCNBCNCCNBBNBBNBBBNBBNBBCBHCBHHNHCBBCBHCB


This polymer grows quickly. After step 5, it has length 97; After step 10, it has length 3073. After step 10, `B` occurs 1749 times, `C` occurs 298 times, `H` occurs 161 times, and `N` occurs 865 times; taking the quantity of the most common element (`B`, 1749) and subtracting the quantity of the least common element (`H`, 161) produces `1749 - 161 = 1588`.

Apply 10 steps of pair insertion to the polymer template and find the most and least common elements in the result. _What do you get if you take the quantity of the most common element and subtract the quantity of the least common element?_

In [1]:
from itertools import pairwise
from collections import Counter, defaultdict


def load(filename):
    with open(filename) as f:
        template = f.readline().strip()
        rules = {}
        for line in f:
            pair, _, insert = line.strip().partition(" -> ")
            if insert:
                rules[pair[0], pair[1]] = pair[0] + insert
    return template, rules


test_data = load("input/day14-test.txt")
real_data = load("input/day14.txt")


def part1(data):
    template, rules = data
    for step in range(10):
        template = "".join(
            [rules.get(pair, pair[0]) for pair in pairwise(template)] + [template[-1]]
        )

    count = Counter(template).most_common()
    return count[0][1] - count[-1][1]


assert part1(test_data) == 1588
print(part1(real_data))

2740

<div class="alert alert-info">For part 1 the string is short enough to just manipulate the data as a string.
</div>

**Your puzzle answer was 2740.**

### Part Two

The resulting polymer isn't nearly strong enough to reinforce the submarine. You'll need to run more steps of the pair insertion process; a total of _40 steps_ should do it.

In the above example, the most common element is `B` (occurring `2192039569602` times) and the least common element is `H` (occurring `3849876073` times); subtracting these produces _`2188189693529`_.

Apply _40_ steps of pair insertion to the polymer template and find the most and least common elements in the result. _What do you get if you take the quantity of the most common element and subtract the quantity of the least common element?_

In [2]:
def part2(data):
    template, rules = data
    rules = {k: v[1] for k, v in rules.items()}

    pair_count = Counter(pairwise(template))
    for step in range(40):
        new_count = Counter()
        for a, b in pair_count:
            if (a, b) in rules:
                c = rules[a, b]
                new_count.update({(a, c): pair_count[a, b], (c, b): pair_count[a, b]})
            else:
                new_count[a, b] += pair_count[a, b]
        pair_count = new_count

    count = defaultdict(int)
    for pair in pair_count:
        count[pair[0]] += pair_count[pair]
    count[template[-1]] += 1
    counts = sorted(count.items(), key=lambda p: p[1], reverse=True)
    return counts[0][1] - counts[-1][1]


assert part2(test_data) == 2188189693529
print(part2(real_data))

2959788056211

<div class="alert alert-info">We can't just calculate the resulting polymer here, so instead each step counts how many
of each pair we get in the next generation. Finally we count the occurences of the first letter of each pair and add on
the final letter (which never changes).
</div>

**Your puzzle answer was 2959788056211.**